# First-order and total Sobol index

In [ ]:
#!pip3 install --upgrade pandas

In [1]:
import numpy as np
import fullmodel_v1_8
import pandas as pd
from Extract_dataV2 import extractData
from SALib.sample import saltelli
from datetime import datetime, timedelta
from multiprocess import Pool, cpu_count

In [2]:
problem = {
    'num_vars': 6,
    'names': ['z_m', 'd_E', 'z_mK_bg', 'p_in', 'P_0', 'Temp'],
    'bounds': [[0, 2.75], 
               [0.02, 0.12], 
               [0.3, 1.0], 
               [0.01, 0.4], 
               [0.01, 0.4], 
               [0, 3.4]]
}
# sample
param_values = saltelli.sample(problem, 2**10)

C:\Users\brian\AppData\Local\Temp\ipykernel_28484\3038930648.py:12: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 2**10)


In [3]:
param_values.shape, (6*2+2)*2**10

((14336, 6), 14336)

In [1]:
param_values[:,5].mean()

NameError: name 'param_values' is not defined

In [9]:
InfoLakes = {'PIGEON LAKE': {'labels': ['MICROCYSTIN, TOTAL',
                              'PHOSPHORUS TOTAL DISSOLVED',
                              'OXYGEN DISSOLVED (FIELD METER)',
                              'Total cyanobacterial cell count (cells/mL)',
                              'TEMPERATURE WATER'],
                              'Years': ['2021'],
                              'DataBase':'merged_water_quality_data.csv',
                              'InitialP':0.05,
                              'InitialO2': 6  },
                
             'PINE LAKE': {'labels': ['MICROCYSTIN, TOTAL',
                              'PHOSPHORUS TOTAL DISSOLVED',
                              'OXYGEN DISSOLVED (FIELD METER)',
                              'Total cyanobacterial cell count (cells/mL)',
                              'TEMPERATURE WATER'],
                              'Years': ['2017'],
                              'DataBase':'merged_water_quality_data.csv',
                              'InitialP':0.4,
                              'InitialO2': 6  },
                             
            'MENDOTA LAKE': {'labels':['Microcystin (nM)',
                                      'OXYGEN DISSOLVED (FIELD METER)',
                                      'Total cyanobacterial cell count (cells/mL)'],
                            'Years':['2018'],
                            'DataBase':'Dataset_US.csv',
                            'InitialP':0.075,
                            'InitialO2': 5 },
                             
            'MONONA LAKE': {'labels':['OXYGEN DISSOLVED (FIELD METER)',
                                          'Total cyanobacterial cell count (cells/mL)',
                                          'TEMPERATURE WATER'],
                            'Years':['2015'],
                            'DataBase':'Combined_Data_for_MO_Merged.csv',
                            'InitialP':0.075,
                            'InitialO2': 5 }}

In [10]:
def model_Sobol_index(z_m,
                      d_E,
                      z_mK_bg,
                      p_in,
                      P_0,
                      Temp, 
                      lake_name = 'PIGEON LAKE'):

    import numpy as np
    import fullmodel_v1_8
    import pandas as pd
    from Extract_dataV2 import extractData
    
    
    # Initial conditions
    M_0 = 0
    A_0 = 0.05
    B_0 = 0.004  # 0.004
    Q_B_0 = 0.01
    Q_A_0 = 0.01
    
    D_0 = 0.0239  # 0.0239
    Y_0 = 0.025*10
    W_0 = 0.0025*10
    
    v_A_0 = 0
    v_D_0 = 0
    v_Y_0 = 0
    v_W_0 = 0
    
    
    # Data
    LakeYear = {'PIGEON LAKE': ['2021'],
                'PINE LAKE': ['2017'],
                'MONONA LAKE': ['2015'],
                'MENDOTA LAKE': ['2018']}
    
    
    years = LakeYear[lake_name]
    
    yearname = ''
    for year in years:
        yearname = yearname + str(year) + '_'
    
    if lake_name == 'PIGEON LAKE' and years[-1] == '2021':
        coment = "_v6_3Var_"
    else:
        coment = "_v6_3Var_" + yearname
    
    
    # # import parameters
    
    if lake_name in ['MONONA LAKE']:
        rectTemp = 3.0
        B_scale = 0.1
        A_scale = 100
        D_scale = 0.1
        P_0 = 0.01
        P_in = 0.01
        O_0 = 5
    
    elif lake_name in ['MENDOTA LAKE']:
        rectTemp = 3.0
        B_scale = 0.1
        A_scale = 10
        D_scale = 10
        P_0 = 0.075
        P_in = 0.015
        O_0 = 5
    
    elif lake_name in ['PIGEON LAKE']:
        rectTemp = 0.0
        B_scale = 1
        A_scale = 1
        D_scale = 1
        P_0 = 0.05
        P_in = 0.01
        O_0 = 6
    
    elif lake_name in ['PINE LAKE']:
        rectTemp = -2.75
        B_scale = 1.0
        A_scale = 2.0
        D_scale = 0.1
        P_0 = 0.4
        P_in = 0.2
        O_0 = 5
    
    
    
    initial_conditions = [M_0, B_0, A_0,
                          Q_B_0,
                          Q_A_0, P_0,
                          D_0,
                          Y_0, W_0,
                          v_A_0, v_D_0,
                          v_Y_0, v_W_0, O_0]
    
    
    def read_params():
        model_CyB = fullmodel_v1_8.modelCyB()
        model_CyB.initial = initial_conditions
        name_data = './FittedParameters/' + \
            'fitting_parameters_full_variables_v2'+lake_name + coment + '_final' + ".csv"
        
        params_fit = pd.read_csv(name_data)
    
        unknow_params = ["alpha_D", "alpha_Y",
                         "tau_D", "tau_Y",
                         "a_A",
                         "sigma_A",
                         "x_A"]
    
        model_CyB.initial[1] = params_fit["B_0"][0]*B_scale
        # print('B(0)=', model_CyB.initial[1])
        model_CyB.initial[2] = params_fit["A_0"][0]*A_scale
        # print('A(0)=', model_CyB.initial[2])
        model_CyB.initial[6] = params_fit["D_0"][0]*D_scale
        # print('D(0)=', model_CyB.initial[6])
    
        for name in unknow_params:
            model_CyB.params[name] = params_fit[name][0]
    
        # Death Daphnia
    
        # 0.015  # 0.03 Pigeon lake  # 0.09601  # 0.125
        model_CyB.params['p_in'] = P_in
        # model_CyB.params["delta_B-"] = 2.4
        # With Toxine
        model_CyB.toxines = True and (model_CyB.initial[1] > 0)
        # New time scale
        # New time scale
        day_start = pd.to_datetime("2023-05-01").day_of_year
    
        days_end = pd.to_datetime("2023-09-30").day_of_year
    
        model_CyB.t_0 = 0  # May 1, 2023
    
        model_CyB.t_f = days_end - day_start
    
        model_CyB.delta_t = model_CyB.t_f*3
    
        model_CyB.set_linetime()
    
        # Temperature and Zm
        path = './ERA5-Land/' + years[-1]
        TempZmData = pd.read_csv(path+lake_name + 'WaterTemperature.csv')
        TempZmData['Date'] = pd.to_datetime(
            TempZmData['Date'], format='mixed')
    
        tempSamp = TempZmData['lake_mix_layer_temperature']-rectTemp
        Zmsample = (TempZmData['lake_mix_layer_depth_min'] +
                    TempZmData['lake_mix_layer_depth_max'])*0.5
        days = TempZmData['Date'].dt.day_of_year
    
        days = np.array(days) - day_start
    
        model_CyB.get_interpTemp(tempSamp, days)
        model_CyB.get_interpZm(Zmsample, days)
        return model_CyB

    model_CyB = read_params()

    
    model_CyB.auxTemp = Temp
    model_CyB.auxZm = z_m

    model_CyB.params['d_E'] = d_E
    model_CyB.params['z_mK_bg'] = z_mK_bg
    model_CyB.params['p_in'] = p_in
    model_CyB.initial[5] = P_0
    
    print(model_CyB.initial[5])
    
    solution, info = model_CyB.solver()
    M_values, B_values, A_values, \
        Q_B_values, Q_A_values, P_values, \
        D_values, Y_values, W_values, \
        v_A_values, v_D_values, v_Y_values, \
        v_W_values, O_values = solution.T
    return B_values

In [11]:
Lakes = ['PIGEON LAKE', 'PINE LAKE', 'MONONA LAKE']
for lake_name in Lakes:
    Yis = []
    NUM_PROCESSES = 6
    pool = Pool(NUM_PROCESSES)
    for i in range(param_values.shape[0]):
        Yi = pool.apply_async(model_Sobol_index, param_values[i,:], {'lake_name': lake_name})
        Yis.append(Yi)
    
    results = np.array([Yi.get() for Yi in Yis])
    # Saving
    df = pd.DataFrame(results)
    yearname = ''
    for year in InfoLakes[lake_name]['Years']:
        yearname = yearname + str(year) + '_'
    path = "./Sobol_Index/"+lake_name + '/'+yearname
    ## Save the DataFrame to a CSV file
    df.to_csv(path + lake_name+'.csv', header=False, index=False)
    print('File is saved in',path , lake_name )

File is saved in ./Sobol_Index/PIGEON LAKE/2021_ PIGEON LAKE
File is saved in ./Sobol_Index/PINE LAKE/2017_ PINE LAKE
File is saved in ./Sobol_Index/MONONA LAKE/2015_ MONONA LAKE
